In [4]:
# i still have to do the one way anovas comparing between WT and MUT and across samples comparing back to HCT WT 


#one-way anovas 
#print bar-graph 

In [44]:
import pandas as pd
import numpy as np
import os 

#my file directory and loading of the data 
data_dir = '/Users/parmeshthakoordial/Desktop/ABIO558Assignment5'

firefly_path = os.path.join(data_dir, 'PT-11nov2025-fireflyLuc-p73muts.xlsx')
nano_path = os.path.join(data_dir, 'PT-11nov2025-nanoLuc-p73muts.xlsx')
plate_map_path = os.path.join(data_dir, 'LuciferaseAssayPlateTemplate11182025.xlsx')

#standardize column name so python knows which row has data 
ROW_HEADER_NAME = 'Row' 

# this code helps reshape data from a plate format to a long format, got from gemini, dont really know how to explain but it converts my data into a betetr format. 

def plate_to_long_format(df, value_name):
    """
    Reshapes wide plate data (e.g., A1, A2...) into long format (Well, Value) 
    using pandas.melt for analysis.
    """
    df = df.rename(columns={df.columns[0]: ROW_HEADER_NAME})
    df.columns = [str(col) if col != ROW_HEADER_NAME else col for col in df.columns]
    
    long_df = pd.melt(
        df, 
        id_vars=[ROW_HEADER_NAME], 
        var_name='Well_Column',     
        value_name=value_name       
    )
    
    long_df['Well'] = long_df[ROW_HEADER_NAME].astype(str) + long_df['Well_Column'].astype(str)
    
    return long_df[['Well', value_name]].dropna(subset=[value_name])


if __name__ == '__main__':
    
    # 1. Import and Reshape Data also from gem
    firefly_df_raw = pd.read_excel(firefly_path, header=0)
    nano_df_raw = pd.read_excel(nano_path, header=0)
    plate_map_df_raw = pd.read_excel(plate_map_path, header=0)

    #code essential to reshape the data to long format
    plate_map_long = plate_to_long_format(plate_map_df_raw, 'Sample_ID')
    firefly_long = plate_to_long_format(firefly_df_raw, 'Luciferase_Value')
    nano_long = plate_to_long_format(nano_df_raw, 'Nano_Value')

    # 2. Merge Data, combining values from nano and firefly excel sheets. 
    merged_df = pd.merge(plate_map_long, firefly_long, on='Well', how='inner') 
    final_df = pd.merge(merged_df, nano_long, on='Well', how='inner')
    
    # 3. Calculate Normalized Ratio (Firefly/Nano)
    # for division by zero numbers to prevent by substituting with very small num 1 nano
    final_df['Nano_Value_Safe'] = np.where(final_df['Nano_Value'] == 0, 1e-9, final_df['Nano_Value'])
    final_df['Normalized_Ratio'] = final_df['Luciferase_Value'] / final_df['Nano_Value_Safe']
    
    # prints the data that I want, selecting the data that i want
    per_well_results = final_df[['Sample_ID', 'Well', 'Luciferase_Value', 'Nano_Value', 'Normalized_Ratio']]

    # Final Output: Print the complete, un-truncated replicate data table.
    print("\n## NORMALIZED REPLICATE DATA (Firefly/Nano)")
    print(per_well_results.round(4).to_string())


## NORMALIZED REPLICATE DATA (Firefly/Nano)
      Sample_ID Well Luciferase_Value Nano_Value Normalized_Ratio
0    HCT116 WT    A1            152.0    4411142         0.000034
1    HCT116 WT    B1            134.0    4359801         0.000031
2    HCT116 WT    C1            141.0    4440581         0.000032
3    HCT116 MUT   F1            261.0    2731872         0.000096
4    HCT116 MUT   G1            397.0    4152924         0.000096
5    HCT116 MUT   H1            200.0    1562676         0.000128
6    DNp63B WT    A2           2532.0     899757         0.002814
7    DNp63B WT    B2           3853.0     759727         0.005072
8    DNp63B WT    C2           2200.0     585231         0.003759
9    DNp63B MUT   F2             10.0      37589         0.000266
10   DNp63B MUT   G2             20.0     117762          0.00017
11   DNp63B MUT   H2             17.0     105346         0.000161
12  DDNp63B WT    A3             57.0     155792         0.000366
13  DDNp63B WT    B3           